In [5]:
!pip install pyspark==3.4.0
bin/kafka-console-consumer --bootstrap-server localhost:9092 --topic topic_streaming_data --from-beginning
bin/kafka-console-consumer.sh --topic topic_streaming_data --bootstrap-server localhost:9092 --from-beginning



[notice] A new release of pip available: 22.2.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [6]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import streaming
import datetime
import os

In [12]:
spark = SparkSession.builder \
    .config("spark.driver.host", "localhost") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.0") \
    .appName("StreamingPipeline") \
    .master("local[2]") \
    .getOrCreate()

The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.

/Users/luiz.oliveira/projects/n_projects/lib/python3.7/site-packages/pyspark/bin/spark-class: line 97: CMD: bad array subscript
head: illegal line count -- -1


RuntimeError: Java gateway process exited before sending its port number

In [3]:
#data_source = "path/to/your/json/data"
#data_source = {x:x for x in range(1000)}

kafka_bootstrap_servers = 'localhost:9092'
kafka_topic = 'topic_streaming_data'

In [4]:
streaming_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
    .option("subscribe", kafka_topic) \
    .load()

In [5]:
streaming_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [23]:
streaming_df = streaming_df.withColumn("year", F.year("timestamp"))
streaming_df = streaming_df.withColumn("month",F.month("timestamp"))
streaming_df = streaming_df.withColumn("day", F.dayofmonth("timestamp"))
streaming_df = streaming_df.withColumn("hour", F.hour("timestamp"))

In [36]:
output_path = os.getcwd().split('stream_pipeline')[0] + "stream_pipeline/output/" + str(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
checkpoint_path = os.getcwd().split('stream_pipeline')[0] + "stream_pipeline/checkpoint_path"

In [40]:
query = streaming_df.writeStream \
    .outputMode("append") \
    .format("parquet") \
    .option("path", output_path) \
    .option("checkpointLocation", checkpoint_path) \
    .partitionBy("year", "month", "day", "hour") \
    .trigger(processingTime='1 minute') \
    .start()

24/01/03 21:44:50 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/01/03 21:44:50 WARN StreamingQueryManager: Stopping existing streaming query [id=547f62bc-88d2-4b31-9c5a-1adba662a352, runId=efa1da5f-ee1c-45a7-818e-456a4ace3687], as a new run is being started.


24/01/03 21:44:50 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


In [38]:
print(query.status)

{'message': 'Waiting for next trigger', 'isDataAvailable': False, 'isTriggerActive': False}


In [47]:
df = spark.read.parquet(os.getcwd().split('stream_pipeline')[0] + "stream_pipeline/output")

In [48]:
df = df.withColumn("value_2", F.decode(F.col("value"), 'UTF-8'))

In [49]:
df.show(truncate=False)

+----+-------------------------------------------------------------------+--------------------+---------+------+-----------------------+-------------+----+-----+---+----+----------------------+
|key |value                                                              |topic               |partition|offset|timestamp              |timestampType|year|month|day|hour|value_2               |
+----+-------------------------------------------------------------------+--------------------+---------+------+-----------------------+-------------+----+-----+---+----+----------------------+
|null|[4D 65 6E 73 61 67 65 6D 20 64 65 20 65 78 65 6D 70 6C 6F 20 30]   |topic_streaming_data|0        |442   |2024-01-03 21:44:44.278|0            |2024|1    |3  |21  |Mensagem de exemplo 0 |
|null|[4D 65 6E 73 61 67 65 6D 20 64 65 20 65 78 65 6D 70 6C 6F 20 31]   |topic_streaming_data|0        |443   |2024-01-03 21:44:44.28 |0            |2024|1    |3  |21  |Mensagem de exemplo 1 |
|null|[4D 65 6E 73 61 67 65 6D

In [ ]:
# Aguardar a conclusão do streaming
query.awaitTermination()